# Social Computing/Social Gaming - Summer 2023
# Exercise Sheet 6 - Transformers and Explainable AI

After we have worked out the base and social model on the previous sheet, we will now take a look into a model with a different approach. This time we will utilize Transformer-based classifiers. Transformers revolutionized NLP field significantly after the paper ["Attention is all your need"](https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) was out. Then, a big [HuggingFace](https://huggingface.co/) platform [2] was created to store and host a lot of opensource NLP pre-trained models. We will work today with some of them.

Furthermore we will make use of SHAP and its underlying Shapley values to understand the basics of neural network explainability. This [site](https://christophm.github.io/interpretable-ml-book/shapley.html) [4] will help you to get an understanding of this concept.

In [18]:
import pandas as pd
import numpy as np

## Task 6.0: The Data

Once again we will use the dataset of Waseem and Hovy [1] as you are already familiar with it and it offers us the possibility to compare it to our previous work.

In [19]:
# Reads the data set from a .csv file
waseem_hovy = pd.read_csv('./tweets.csv')
waseem_hovy = waseem_hovy.astype(str)

# This drop operation is necessary because of an inconsistency in the dataset
waseem_hovy = waseem_hovy.drop([3343, 3344])
waseem_hovy = waseem_hovy[['text', 'label']]

# We need to do a unique and precise reordering to match with graph information later on
unique_tweets, indices = np.unique(waseem_hovy['text'].to_numpy(), return_index=True)
ordered_labels = waseem_hovy['label'].to_numpy()[indices]
waseem_hovy = pd.DataFrame(np.stack((unique_tweets, ordered_labels), axis=1), columns=['text', 'label'])

/var/folders/qd/q9r9nqmx6_931g011m9qv1xw0000gn/T/ipykernel_74264/3071680186.py:2: DtypeWarning: Columns (2,30) have mixed types. Specify dtype option on import or set low_memory=False.
  waseem_hovy = pd.read_csv('./tweets.csv')


## Task 6.1: Preprocessing

### Encode the labels
Since we are using the same dataset, we need to covert their textual representation into numerical. For this task (almost the same as for a previous sheet), we can use [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
# Extract labels from dataset

original_labels = np.array(waseem_hovy["label"].tolist())

# DONE:
data_labels = LabelEncoder().fit_transform(original_labels)

# Shows the actual shape of the labels
print(original_labels.shape)
print(np.unique(original_labels))
print(data_labels.shape)
print(np.unique(data_labels))

(16849,)
['none' 'racism' 'sexism']
(16849,)
[0 1 2]


In the case of transforfer usage, we do not need to preprocess and tokenize sentences by ourselves -- that will be done by model's tokenizer! So, let us dive into transfermers right now!

## BERT for Toxic speech classification

We will fine-tune to our downstream task BERT model [3], more specifically -- distilled version of it [DistilBERT](https://huggingface.co/distilbert-base-uncased).

But, firstly, we need to install [transformers](https://huggingface.co/learn/nlp-course/chapter2/1?fw=pt) library:

In [22]:
from transformers import BertTokenizer, BertForSequenceClassification

In [23]:
# DONE: initialize tokenizer and model for DistilBERT or
# for other model of your preferences -- you are very welcome to try out something different!

tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['vocab_projector.bias', 'distilbert.transformer.layer.1.attention.q_lin.weight', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.3.ffn.lin2.weight', 'distilbert.transformer.layer.1.attention.v_lin.weight', 'distilbert.transformer.layer.1.attention.v_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.1.ffn.lin1.bias', 'distilbert.embeddings.wo

### Training Batches Preparation

The same as in the previous tutorial, we will create our custom datasets and loaders to generate batches for training. However, we need to adapt it to transformers input:

1.   Each dataset item should return ``input_ids``, ``attention_mask``, and ``label``.
2.   All should be ``tensors``.
3.   In the end, you need to apply ``collate_fn`` -- that will pad all tensors in batches to the max_length (already implemented for you).



In [24]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [25]:
# this is a special function that pad sequences in the batch for data loader

def collate_fn(batch):
    input_ids = torch.nn.utils.rnn.pad_sequence([item['input_ids'] for item in batch], batch_first=True)
    attention_mask = torch.nn.utils.rnn.pad_sequence([item['attention_mask'] for item in batch], batch_first=True)
    labels = torch.stack([item['label'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label': labels
    }

In [26]:
# DONE: create your CustomDataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        # DONE
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        # DONE
        return len(self.labels)

    def __getitem__(self, index):
        # DONE
        encoding = self.tokenizer(
            self.texts[index],
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(self.labels[index])
        }

# ––––––––––––––– End of Solution –––––––––––––––––––

In [27]:
from sklearn.model_selection import train_test_split

texts = waseem_hovy['text'].values

# DONE: Split tweets and labels in Train/Test/Validation 60/20/20
X_train, temp_texts, y_train, temp_labels = train_test_split(texts, data_labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

print("Training data shape: {}, Labels shape: {}".format(X_train.shape, y_train.shape))
print("Test data shape: {}, Labels shape: {}".format(X_test.shape, y_test.shape))
print("Validation data shape: {}, Labels shape: {}".format(X_val.shape, y_val.shape))

Training data shape: (10109,), Labels shape: (10109,)
Test data shape: (3370,), Labels shape: (3370,)
Validation data shape: (3370,), Labels shape: (3370,)


In [28]:
BATCH_SIZE = 32

# Create the Datasets
train_dataset = CustomDataset(X_train, y_train, tokenizer)
val_dataset = CustomDataset(X_val, y_val, tokenizer)
test_dataset = CustomDataset(X_test, y_test, tokenizer)

# DataLoader for batching and parallel data loading
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

### Training Loop

We are ready to train the model! You can reuse the code from the previous tutorial:
1. Define ``optimizer`` and ``criterion`` for a classiication task.
2. Use ``train_lodaer`` to sample batches for training.
3. Track validation loss using data from ``val_loader``.
4. Achtung: now the items in batches have different structure!

In [29]:
import torch.nn as nn

In [30]:
# DONE: define optimizer and criterion

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device);

In [35]:
%%time

# DONE: implement training loop

# you can play with different number of epochs and check the model's performance!
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for iteration, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{iteration+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.4f}")
        
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        
        for iteration, batch in enumerate(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            predicted_labels = torch.argmax(outputs, dim=1)
            
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)

         accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.4f}')

print("Training complete!")

###

Epoch [1/3], Step [1/316], Loss: 0.7053
Epoch [1/3], Step [2/316], Loss: 0.7314
Epoch [1/3], Step [3/316], Loss: 0.9537
Epoch [1/3], Step [4/316], Loss: 1.1709
Epoch [1/3], Step [5/316], Loss: 0.8197
Epoch [1/3], Step [6/316], Loss: 0.7228
Epoch [1/3], Step [7/316], Loss: 0.7301
Epoch [1/3], Step [8/316], Loss: 0.7684
Epoch [1/3], Step [9/316], Loss: 0.8017
Epoch [1/3], Step [10/316], Loss: 0.7866
Epoch [1/3], Step [11/316], Loss: 0.8603
Epoch [1/3], Step [12/316], Loss: 0.6776
Epoch [1/3], Step [13/316], Loss: 1.0044
Epoch [1/3], Step [14/316], Loss: 0.9233
Epoch [1/3], Step [15/316], Loss: 0.6710
Epoch [1/3], Step [16/316], Loss: 0.7234
Epoch [1/3], Step [17/316], Loss: 0.7442
Epoch [1/3], Step [18/316], Loss: 0.8716
Epoch [1/3], Step [19/316], Loss: 0.7289
Epoch [1/3], Step [20/316], Loss: 0.6748
Epoch [1/3], Step [21/316], Loss: 0.9503
Epoch [1/3], Step [22/316], Loss: 0.9624
Epoch [1/3], Step [23/316], Loss: 0.6941
Epoch [1/3], Step [24/316], Loss: 0.7773
Epoch [1/3], Step [25/316

In [36]:
# TODO: Evaluate the model based on test_loader

# Evaluation loop
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)
    
    accuracy = total_correct / total_samples
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8347


## SHAP Explanations

Now, we have a decent model for toxic speech detection. However, sometimes it can be not so clear why some sample is considered toxic or not. We can try to explaine the model's decision! For this, we utilize [SHAP](https://shap.readthedocs.io/en/latest/index.html).

In [ ]:
# example of module installation

# !pip install shap

In [ ]:
#import sys
#sys.path.append('/content/drive/MyDrive/tum/tum_exercises/Ex6_XAI_original/src')
#sys.path.append('/content/drive/MyDrive/tum/tum_exercises/Ex6_XAI_original/src/evaluation.py')

NumPy version: 1.25.0


In [ ]:
import matplotlib.pyplot as plt
import shap
from src.explainability import shap_explain_text

# Initializes JavaScript to visualize plots generated by Shap
shap.initjs()


### Let's inspect our model!

#### Easy initialization and exploration of a cherry-picked sample.

In [ ]:
import transformers

In [ ]:
# We need to create a text-classification pipeline to input it to the explainer

pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, return_all_scores=True)

In [ ]:
# This is the example how to define Explainer from SHAP

explainer = shap.Explainer(pred)
text = ['only men can have higher education']
shap_values = explainer(text)

In [ ]:
shap.plots.text(shap_values)

In addition to slicing, Explanation objects also support a set of reducing methods. Here we use the ``.mean(0)`` to take the average impact of all words towards the “sexism” (``idx=2``) class. Note that here we are also averaging over three examples, to get a better summary you would want to use a larger portion of the dataset.

In [ ]:
shap.plots.text(shap_values[:, :, 2])

### We can also work with already fine-tuned for sexism classification models.

There are already different models fine-tuned for toxic or sexism speech detection. You can load, for instance, our model [bertweet-sexism](https://huggingface.co/tum-nlp/bertweet-sexism) and try to explaine it as well!

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# TODO: define tokenizer and model for bertweet-sexism instance.

###

Here a small example on cherry-picked sentence to explaine the model:

In [ ]:
# TODO: inspect the model on the sample!

**TODO: how can we understand if some model is better or not?**
You can pick the misclassified samples for each model and compare the explanations. Can you explaine why the model did mistakes? Are these mistakes the same or not? Try out other ways how to use SHAP [here](https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/sentiment_analysis/Emotion%20classification%20multiclass%20example.html). Maybe, other ways of explanations can be useful more?

**TODO: Write your observations here**

**Concluding questions:**
* Do the hate scores perform as expected in our model?
* If not, can you come up with a possible explanation for that even though the models with social scores performed better?
* What does that tell you about applicability of neural networks and their trustworthiness?

**TODO: Write your answers here**

### <center> Thank you for participating in Social Computing/Social Gaming 2023. </center>
### <center> Good luck with the exams! </center>

## References

[1] Waseem, Z., & Hovy, D. (2016). Hateful symbols or hateful people? Predictive features for hate speech detection on Twitter. In Proceedings of the naacl student research workshop (pp. 88-93).
<br> [2] [HuggingFace Tutorial](https://huggingface.co/learn/nlp-course/chapter1/1)
<br> [3] [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)
<br> [4] https://christophm.github.io/interpretable-ml-book/shapley.html